In [ ]:
pip install keras-utils

In [ ]:
import cv2
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras_utils
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tqdm import tqdm

In [ ]:
traindf = pd.read_csv('../input/dataset/traindf.csv')
traindf

#### reading train images

In [ ]:
path = "../input/siim-covid19-resized-to-256px-jpg/train"
imgs=os.listdir(path)

In [ ]:
images = []
for i in range(traindf.shape[0]):
    kk = traindf[i:i+1]
    #print(kk)
    img = kk['id']
    img = img[i]
    imgf = img + '.jpg'
    imgfile = os.path.join(path,imgf)
    #print(imgfile)
    imgg = cv2.imread(imgfile)
    images.append(imgg)

#### Study Train

In [ ]:
studyTrain = traindf.drop(["Unnamed: 0","boxes", "label", "Olabel", "integer_Olabel"],axis=1)
studyTrain

##### preparing data for modeling

In [ ]:
images=np.array(images)
images.shape

In [ ]:
y_study=studyTrain['integer_label'].values
y_study = to_categorical(y_study)
y_study

In [ ]:
y_study.shape

In [ ]:
plt.imshow(images[10])
y_study[10]

#### CNN modelling for studyTrain data

In [ ]:
import keras
from keras import layers
from keras import Sequential
from keras.layers import Dense


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, y_study, random_state=42, test_size=0.2)

In [ ]:
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(256,256,3)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(4, activation='softmax'))

In [ ]:
model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
model1.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

#### Image Train

In [ ]:
imageTrain = traindf.drop(["Unnamed: 0", "Negative", "Typical", "Indeterminate", "Atypical", "label_y", "integer_label"],axis=1)
imageTrain

##### preparing data for modeling

images=np.array(images)
images.shape

In [ ]:
y_image=imageTrain['integer_Olabel'].values
y_image = to_categorical(y_image)

In [ ]:
y_image

In [ ]:
y_image.shape

In [ ]:
plt.imshow(images[10])
y_image[10]

#### CNN modeling for imageTrain data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, y_image, random_state=42, test_size=0.2)

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(256,256,3)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(2, activation='sigmoid'))

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss = 'binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
model2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

#### reading test data

In [ ]:
testdf = pd.read_csv('../input/testdataset/test.csv')
testdf

In [ ]:
testdf.drop(["Unnamed: 0"], axis = 1)

#### reading test images

In [ ]:
pathtest = "../input/siim-covid19-resized-to-256px-jpg/test"
imgs=os.listdir(pathtest)

In [ ]:
testimages = []
for i in range(testdf.shape[0]):
    kk = testdf[i:i+1]
    #print(kk)
    img = kk['image']
    img = img[i]
    imgf = img + '.jpg'
    imgfile = os.path.join(pathtest,imgf)
    #print(imgfile)
    imgg = cv2.imread(imgfile)
    testimages.append(imgg)

#### Predictions on study level

In [ ]:
testStudy=np.array(testimages)
testStudy.shape

In [ ]:
prediction = model1.predict_classes(testStudy)

In [ ]:
sampleStudy = pd.read_csv('../input/samplesubmissions/Studysubmission.csv')
sampleStudy

In [ ]:
sampleStudy = sampleStudy.drop(["Unnamed: 0"], axis = 1)

In [ ]:
sampleStudy['label'] = prediction

In [ ]:
sampleStudy

In [ ]:
sampleStudy.loc[sampleStudy['label']==3, 'PredictionString'] = 'typical 1 0 0 1 1'
sampleStudy.loc[sampleStudy['label']==0, 'PredictionString'] = 'atypical 1 0 0 1 1'
sampleStudy.loc[sampleStudy['label']==1, 'PredictionString'] = 'indeterminate 1 0 0 1 1'

In [ ]:
sampleStudy

In [ ]:
sampleStudy = sampleStudy.drop(["id_image", "label"], axis = 1)
sampleStudy

#### Predictions on image level

In [ ]:
testImage=np.array(testimages)
testImage.shape

In [ ]:
prediction = model2.predict_classes(testImage)

In [ ]:
sampleImage = pd.read_csv('../input/samplesubmissions/Imagesubmission.csv')
sampleImage

In [ ]:
sampleImage = sampleImage.drop(["Unnamed: 0"], axis = 1)

In [ ]:
sampleImage['label'] = prediction
sampleImage

In [ ]:
sampleImage.loc[sampleImage['label']==1, 'PredictionString'] = 'opacity 1 0 0 1 1'

In [ ]:
sampleImage = sampleImage.drop(["label"], axis = 1)
sampleImage

#### Creating the submission file

In [ ]:
submission = pd.concat([sampleStudy, sampleImage], sort = False, ignore_index = True)
submission

In [ ]:
submission.to_csv('./submission.csv', index=False)